##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cuantización entera postentrenamiento

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/performance/post_training_integer_quant"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

## Visión general

La cuantización entera es una estrategia de optimización que convierte los números de punto flotante de 32 bits (como las ponderaciones y las salidas de activación) en los números de punto fijo de 8 bits más próximos. Esto resulta en un modelo más pequeño y una mayor velocidad de inferencia, lo que es valioso para dispositivos de bajo consumo como [microcontroladores](https://www.tensorflow.org/lite/microcontrollers). Este formato de datos también es requerido por aceleradores de sólo enteros como el [Edge TPU](https://coral.ai/).

En este tutorial, entrenará un modelo MNIST desde cero, lo convertirá en un archivo Tensorflow Lite y lo cuantizará usando [cuantización postentrenamiento](https://www.tensorflow.org/lite/performance/post_training_quantization). Por último, revisará la precisión del modelo convertido y lo comparará con el modelo flotante original.

En realidad, tiene varias opciones sobre cuánto desea cuantizar un modelo. En este tutorial, realizará una "cuantización entera completa", que convierte todas las ponderaciones y salidas de activación en datos enteros de 8 bits, mientras que otras estrategias pueden dejar cierta cantidad de datos en punto flotante.

Para saber más sobre las distintas estrategias de cuantización, lea sobre [Optimización del modelo TensorFlow Lite](https://www.tensorflow.org/lite/performance/model_optimization).


## Preparación

Para cuantizar tanto los tensores de entrada como los de salida, necesitamos usar las API añadidas en TensorFlow 2.3:

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
print("TensorFlow version: ", tf.__version__)

## Generar un modelo TensorFlow

Construiremos un modelo sencillo para clasificar números del conjunto de datos [MNIST](https://www.tensorflow.org/datasets/catalog/mnist).

Este entrenamiento no le llevará mucho tiempo porque está entrenando el modelo durante sólo 5 épocas, lo que entrena con una precisión aproximada del ~98%.

In [ ]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Define the model architecture
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)

## Convertir a un modelo TensorFlow Lite

Ahora puede convertir el modelo entrenado al formato TensorFlow Lite utilizando el [Convertidor](https://www.tensorflow.org/lite/models/convert) de TensorFlow Lite, y aplicar diversos grados de cuantización.

Tenga en cuenta que algunas versiones de la cuantización dejan parte de los datos en formato float. Así que las siguientes secciones muestran cada opción con cantidades crecientes de cuantización, hasta que obtenemos un modelo que es enteramente datos int8 o uint8 (tenga en cuenta que duplicamos algo de código en cada sección para que pueda ver todos los pasos de cuantización de cada opción).

En primer lugar, aquí tiene un modelo convertido sin cuantización:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

Ahora es un modelo TensorFlow Lite, pero sigue usando valores flotantes de 32 bits para todos los datos de los parámetros.

### Convertir usando cuantización de rango dinámico


Activemos ahora el indicador predeterminado `optimizaciones` para cuantificar todos los parámetros fijos (como las ponderaciones):

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()

El modelo es ahora un poco más pequeño con ponderaciones cuantificadas, pero los demás datos variables siguen estando en formato flotante.

### Convertir usando la cuantización de reserva flotante

Para cuantizar los datos variables (como la entrada/salida del modelo y los intermedios entre capas), debe dar un [`RepresentativeDataset`](https://www.tensorflow.org/api_docs/python/tf/lite/RepresentativeDataset). Se trata de una función generadora que ofrece un conjunto de datos de entrada lo suficientemente grande como para representar valores típicos. Permite al convertidor estimar un rango dinámico para todos los datos variables (no es necesario que el conjunto de datos sea único en comparación con el conjunto de datos de entrenamiento o evaluación). Para admitir entradas múltiples, cada punto de datos representativo es una lista y los elementos de la lista se introducen en el modelo según sus índices.


In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()

Ahora todas las ponderaciones y los datos variables están cuantizados, y el modelo es significativamente más pequeño en comparación con el modelo original de TensorFlow Lite.

Sin embargo, para mantener la compatibilidad con las aplicaciones que tradicionalmente usan tensores de entrada y salida del modelo en float, el conversor TensorFlow Lite deja los tensores de entrada y salida del modelo en float:

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

Eso suele ser bueno para la compatibilidad, pero no será compatible con dispositivos que sólo realizan operaciones basadas en números enteros, como el TPU Edge.

Además, el proceso anterior puede dejar una operación en formato flotante si TensorFlow Lite no incluye una implementación cuantizada para esa operación. Esta estrategia permite que se llene la conversión para que tenga un modelo más pequeño y eficiente, pero de nuevo, no será compatible con hardware de sólo enteros (todas las ops de este modelo MNIST tienen una implementación cuantizada).

Por tanto, para garantizar un modelo de sólo enteros de principio a fin, necesita un par de parámetros más...

### Convertir usando cuantización de sólo enteros

Para cuantizar los tensores de entrada y salida, y hacer que el conversor lance un error si encuentra una operación que no puede cuantizar, convierta de nuevo el modelo con algunos parámetros adicionales:

In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

La cuantización interna sigue siendo la misma que antes, pero puede ver que los tensores de entrada y salida tienen ahora formato entero:


In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

Ahora tiene un modelo cuantizado entero que usa datos enteros para los tensores de entrada y salida del modelo, por lo que es compatible con hardware de sólo enteros como el [Edge TPU](https://coral.ai).

### Guardar los modelos como archivos

Necesitará un archivo `.tflite` para implementar su modelo en otros dispositivos. Así que a continuación vamos a guardarlos modelos convertidos en archivos y luego cargarlos cuando ejecutemos inferencias.

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)
# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"mnist_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

## Ejecutar los modelos TensorFlow Lite

Ahora ejecutaremos inferencias usando el `Intérprete` de TensorFlow Lite para comparar las precisiones del modelo.

En primer lugar, necesitamos una función que ejecute la inferencia con un modelo y unas imágenes dadas y, a continuación, devuelva las predicciones:


In [ ]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_images[test_image_index]
    test_label = test_labels[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions


### Probar los modelos en una imagen


Ahora compararemos el rendimiento del modelo flotante y del modelo cuantizado:

- `tflite_model_file` es el modelo original de TensorFlow Lite con datos de punto flotante.
- `tflite_model_quant_file` es el último modelo que convertimos usando cuantización sólo de enteros (usa datos uint8 para la entrada y la salida).

Creemos otra función para mostrar nuestras predicciones:

In [ ]:
import matplotlib.pylab as plt

# Change this to test a different image
test_image_index = 1

## Helper function to test the models on one image
def test_model(tflite_file, test_image_index, model_type):
  global test_labels

  predictions = run_tflite_model(tflite_file, [test_image_index])

  plt.imshow(test_images[test_image_index])
  template = model_type + " Model \n True:{true}, Predicted:{predict}"
  _ = plt.title(template.format(true= str(test_labels[test_image_index]), predict=str(predictions[0])))
  plt.grid(False)

Ahora pruebe el modelo flotante:

In [ ]:
test_model(tflite_model_file, test_image_index, model_type="Float")

Y pruebe el modelo cuantizado:

In [ ]:
test_model(tflite_model_quant_file, test_image_index, model_type="Quantized")

### Evalúe los modelos en todas las imágenes

Ahora vamos a ejecutar ambos modelos usando todas las imágenes de prueba que cargamos al principio de este tutorial:

In [ ]:
# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_images
  global test_labels

  test_image_indices = range(test_images.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_images)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_images)))

Evalúe el modelo flotante:

In [ ]:
evaluate_model(tflite_model_file, model_type="Float")

Evalúe el modelo cuantizado:

In [ ]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Así que ahora tiene un modelo cuantizado entero con casi ninguna diferencia en la precisión, en comparación con el modelo flotante.

Para saber más sobre otras estrategias de cuantización, lea sobre [Optimización del modelo TensorFlow Lite](https://www.tensorflow.org/lite/performance/model_optimization).